<a href="https://colab.research.google.com/github/KevinTheRainmaker/Speech_Emotion_Classifier/blob/main/SpeechEmotionClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np

import os
import sys

# to analyze audio
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [7]:
# mount gdrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path = '/content/drive/MyDrive/data/emotional_speech'

Mounted at /content/drive


In [34]:
# paths
crema = os.path.join(path, "CREMA-D")
savee = os.path.join(path, "SAVEE")
ravdess = os.path.join(path, "RAVDESS")
tess = os.path.join(path, "TESS")

## Data Preperation

### 1. CREMA Dataframe

In [17]:
crema_directory_list = os.listdir(crema)

In [21]:
file_emotion = []
file_path = []

for file in crema_directory_list:
    # storing file paths
    file_path.append(crema + file)
    # storing file emotions
    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')

In [22]:
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
emotion_df.head()

,Emotions
0,sad
1,sad
2,fear
3,sad
4,disgust


In [23]:
# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
crema_df = pd.concat([emotion_df, path_df], axis=1)
crema_df.head()

,Emotions,Path
0,sad,/content/drive/MyDrive/data/emotional_speech/C...
1,sad,/content/drive/MyDrive/data/emotional_speech/C...
2,fear,/content/drive/MyDrive/data/emotional_speech/C...
3,sad,/content/drive/MyDrive/data/emotional_speech/C...
4,disgust,/content/drive/MyDrive/data/emotional_speech/C...


### 2. SAVEE Dataset

In [24]:
savee_directory_list = os.listdir(savee)

In [26]:
file_emotion = []
file_path = []

for file in savee_directory_list:
    file_path.append(savee + file)
    part = file.split('_')[1]
    ele = part[:-6]
    if ele=='a':
        file_emotion.append('angry')
    elif ele=='d':
        file_emotion.append('disgust')
    elif ele=='f':
        file_emotion.append('fear')
    elif ele=='h':
        file_emotion.append('happy')
    elif ele=='n':
        file_emotion.append('neutral')
    elif ele=='sa':
        file_emotion.append('sad')
    else:
        file_emotion.append('surprise')

In [28]:
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
emotion_df.head()

,Emotions
0,neutral
1,surprise
2,neutral
3,sad
4,sad


In [30]:
path_df = pd.DataFrame(file_path, columns=['Path'])
savee_df = pd.concat([emotion_df, path_df], axis=1)
savee_df.head()

,Emotions,Path
0,neutral,/content/drive/MyDrive/data/emotional_speech/S...
1,surprise,/content/drive/MyDrive/data/emotional_speech/S...
2,neutral,/content/drive/MyDrive/data/emotional_speech/S...
3,sad,/content/drive/MyDrive/data/emotional_speech/S...
4,sad,/content/drive/MyDrive/data/emotional_speech/S...


### 3. RAVDESS Dataframe

**Filename identifiers**

- Modality (01 = full-AV, 02 = video-only, 03 = audio-only).

- Vocal channel (01 = speech, 02 = song).

- Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).

- Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.

- Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").

- Repetition (01 = 1st repetition, 02 = 2nd repetition).

- Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

In [57]:
ravdess_directory_list = os.listdir(ravdess)

In [60]:
file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    actor = os.listdir(ravdess + '/' + dir)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        # print(part[2])
        file_emotion.append(int(part[2]))
        file_path.append(ravdess + dir + '/' + file)

In [61]:
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
emotion_df.head()

,Emotions
0,7
1,4
2,8
3,8
4,4


In [62]:
path_df = pd.DataFrame(file_path, columns=['Path'])
ravdess_df = pd.concat([emotion_df, path_df], axis=1)
ravdess_df.head()

,Emotions,Path
0,7,/content/drive/MyDrive/data/emotional_speech/R...
1,4,/content/drive/MyDrive/data/emotional_speech/R...
2,8,/content/drive/MyDrive/data/emotional_speech/R...
3,8,/content/drive/MyDrive/data/emotional_speech/R...
4,4,/content/drive/MyDrive/data/emotional_speech/R...


In [63]:
# changing integers to actual emotions.
ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
ravdess_df.head()

,Emotions,Path
0,disgust,/content/drive/MyDrive/data/emotional_speech/R...
1,sad,/content/drive/MyDrive/data/emotional_speech/R...
2,surprise,/content/drive/MyDrive/data/emotional_speech/R...
3,surprise,/content/drive/MyDrive/data/emotional_speech/R...
4,sad,/content/drive/MyDrive/data/emotional_speech/R...


### 4. TESS Dataset

In [47]:
tess_directory_list = os.listdir(tess)

In [48]:
file_emotion = []
file_path = []

for dir in tess_directory_list:
    directories = os.listdir(tess + '/' + dir)
    # print(directories)
    for file in directories:
        part = file.split('.')[0]
        part = part.split('_')[2]
        if part=='ps':
            file_emotion.append('surprise')
        else:
            file_emotion.append(part)
        file_path.append(tess + dir + '/' + file)

In [49]:
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
emotion_df.head()

,Emotions
0,sad
1,sad
2,sad
3,sad
4,sad


In [50]:
path_df = pd.DataFrame(file_path, columns=['Path'])
tess_df = pd.concat([emotion_df, path_df], axis=1)
tess_df.head()

,Emotions,Path
0,sad,/content/drive/MyDrive/data/emotional_speech/T...
1,sad,/content/drive/MyDrive/data/emotional_speech/T...
2,sad,/content/drive/MyDrive/data/emotional_speech/T...
3,sad,/content/drive/MyDrive/data/emotional_speech/T...
4,sad,/content/drive/MyDrive/data/emotional_speech/T...
